In [3]:
import re    # for regular expressions 
import nltk  # for text manipulation 
import string # for text manipulation 
import numpy as np 
import pandas as pd # for data manipulation 
import matplotlib.pyplot as plt
import pickle

Reading the CSV file

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/tweets.csv",encoding='ISO-8859-1')
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "TweetText"]
data.columns = DATASET_COLUMNS
data.head()

,target,ids,date,flag,user,TweetText
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Droping all the unnecessary features

In [ ]:
data.drop(['ids','date','flag','user'],axis = 1,inplace = True)
data.head()

,target,TweetText
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [ ]:
positif_data = data[data.target==4].iloc[:50000,:]
print(positif_data.shape)
negative_data = data[data.target==0].iloc[:50000,:]
print(negative_data.shape)

(50000, 2)
(50000, 2)


In [ ]:
data = pd.concat([positif_data,negative_data],axis = 0)
print(data.shape)
data.head()

(100000, 2)


,target,TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!
800000,4,im meeting up with one of my besties tonight! ...
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,4,Being sick can be really cheap when it hurts t...
800003,4,@LovesBrooklyn2 he has that effect on everyone


removing all the usename (Start's with @)

In [ ]:
data['Clean_TweetText'] = data['TweetText'].str.replace("@", "") 
data.head()

,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,I LOVE Health4UandPets u guys r the best!!
800000,4,im meeting up with one of my besties tonight! ...,im meeting up with one of my besties tonight! ...
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...","DaRealSunisaKim Thanks for the Twitter add, Su..."
800002,4,Being sick can be really cheap when it hurts t...,Being sick can be really cheap when it hurts t...
800003,4,@LovesBrooklyn2 he has that effect on everyone,LovesBrooklyn2 he has that effect on everyone


Removing all the links

In [ ]:
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "") 
data.head()

<ipython-input-9-2ad241b0fe75>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "")


,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,I LOVE Health4UandPets u guys r the best!!
800000,4,im meeting up with one of my besties tonight! ...,im meeting up with one of my besties tonight! ...
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...","DaRealSunisaKim Thanks for the Twitter add, Su..."
800002,4,Being sick can be really cheap when it hurts t...,Being sick can be really cheap when it hurts t...
800003,4,@LovesBrooklyn2 he has that effect on everyone,LovesBrooklyn2 he has that effect on everyone


Allowing only Characters

In [ ]:
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ") 
data.head()

<ipython-input-10-1125870a2303>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ")


,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,I LOVE Health UandPets u guys r the best
800000,4,im meeting up with one of my besties tonight! ...,im meeting up with one of my besties tonight ...
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...",DaRealSunisaKim Thanks for the Twitter add Su...
800002,4,Being sick can be really cheap when it hurts t...,Being sick can be really cheap when it hurts t...
800003,4,@LovesBrooklyn2 he has that effect on everyone,LovesBrooklyn he has that effect on everyone


importint nltk to remove all the stopwords

In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords])
    return clean_text

In [ ]:
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda text : remove_stopwords(text.lower()))
data.head()

,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,love health uandpets u guys r best
800000,4,im meeting up with one of my besties tonight! ...,im meeting one besties tonight cant wait girl ...
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...",darealsunisakim thanks twitter add sunisa got ...
800002,4,Being sick can be really cheap when it hurts t...,sick really cheap hurts much eat real food plu...
800003,4,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyone


In [ ]:
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: x.split())
data.head()

,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,"[love, health, uandpets, u, guys, r, best]"
800000,4,im meeting up with one of my besties tonight! ...,"[im, meeting, one, besties, tonight, cant, wai..."
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...","[darealsunisakim, thanks, twitter, add, sunisa..."
800002,4,Being sick can be really cheap when it hurts t...,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,4,@LovesBrooklyn2 he has that effect on everyone,"[lovesbrooklyn, effect, everyone]"


Stemming

In [ ]:
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: [stemmer.stem(i) for i in x])
data.head()

,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,"[love, health, uandpet, u, guy, r, best]"
800000,4,im meeting up with one of my besties tonight! ...,"[im, meet, one, besti, tonight, cant, wait, gi..."
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...","[darealsunisakim, thank, twitter, add, sunisa,..."
800002,4,Being sick can be really cheap when it hurts t...,"[sick, realli, cheap, hurt, much, eat, real, f..."
800003,4,@LovesBrooklyn2 he has that effect on everyone,"[lovesbrooklyn, effect, everyon]"


In [ ]:
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x]))
data.head()

,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,love health uandpet u guy r best
800000,4,im meeting up with one of my besties tonight! ...,im meet one besti tonight cant wait girl talk
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...",darealsunisakim thank twitter add sunisa got m...
800002,4,Being sick can be really cheap when it hurts t...,sick realli cheap hurt much eat real food plu ...
800003,4,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyon


Removing all the small words

In [ ]:
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data.head()

,target,TweetText,Clean_TweetText
799999,4,I LOVE @Health4UandPets u guys r the best!!,love health uandpet best
800000,4,im meeting up with one of my besties tonight! ...,meet besti tonight cant wait girl talk
800001,4,"@DaRealSunisaKim Thanks for the Twitter add, S...",darealsunisakim thank twitter sunisa meet show...
800002,4,Being sick can be really cheap when it hurts t...,sick realli cheap hurt much real food friend m...
800003,4,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyon


Modeling

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Vectorizing

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english') 
cv = count_vectorizer.fit_transform(data['Clean_TweetText'])
cv.shape

(100000, 65027)

50000

Splitting

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(cv,data['target'] , test_size=.2,stratify=data['target'], random_state=42)


In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
prediction_lr = lr.predict(X_test)
print(accuracy_score(prediction_lr,y_test))

0.7352


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
svc = svm.SVC()
svc.fit(X_train,y_train)
prediction_svc = svc.predict(X_test)
print(accuracy_score(prediction_svc,y_test))

0.73805


In [ ]:
prediction_svc

array([4, 4, 0, ..., 4, 0, 0])

In [ ]:
import pickle
pickle.dump(count_vectorizer,open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/transform.pkl','wb'))

In [ ]:
pickle.dump(svc,open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/svc.pkl','wb'))

In [ ]:
pickle.dump(lr,open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/logisticregression.pkl','wb'))

In [ ]:
pickle.dump(stemmer,open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/stem.pkl','wb'))

In [ ]:
# 0-> depression
# 0->not-depression
# Everything is depressed
# just need to implement front-end part

In [19]:
stemmer= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/stem.pkl','rb'))
count_vectorizer= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/transform.pkl','rb'))
l= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/logisticregression.pkl','rb'))

In [26]:
svc= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Depressed Tweet Analysis/models/svc.pkl','rb'))

EOFError: ignored

In [20]:
def clean(data):
  data.replace("@","")
  data.replace(r"http\S+", "")
  data.replace("[^a-zA-Z]", " ")
  data=remove_stopwords(data)
  data=data.lower()
  data=data.split()
  temp=[]
  for word in data:
    if len(word)>=3:
      temp.append(stemmer.stem(word))
  data=temp
  vect=count_vectorizer.transform(data).toarray()
  ans=l.predict(vect)
  return ans



In [21]:
def calculate(msg):
  vect=clean(msg)
  depress=0
  non_depress=1
  for val in vect:
    if val == 0:
      depress+=1
    else:
      non_depress+=1
  return (depress,non_depress)

In [24]:
(depress,non_depress)=calculate("I am very depression depression  in my life ..")
per_ndepress=depress/(depress+non_depress)
if(per_ndepress >=0.35):
  print("depressed")
elif(per_ndepress>=0.15 and per_ndepress<=0.3):
  print("Neutral")
else:
  print("postive")

depressed
